In [57]:
import torch.nn as nn
import torch
from torch.nn import functional as F
import pandas as pd
import math

In [2]:
file_loc = 'data/tiny-shakespeare.txt'
# Open the file in read mode
with open(file_loc, 'r') as file:
    # Read the contents of the file
    content = file.read()

# Print the content
print(content[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
char_set = sorted(set(content))
encode_dict = {}
decode_dict = {}
for index, char in enumerate(char_set):
    encode_dict[char] = index
    decode_dict[index] = char
def encode(text):
    return [encode_dict[ch] for ch in text]
def decode(tokens):
    return ''.join([decode_dict[token] for token in tokens])
vocab_size = len(char_set)
#test
decode(encode('hi there'))

'hi there'

In [4]:
data_tensor = torch.tensor(encode(content))
data_tensor[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [5]:
train_ratio = 0.9
train_size = int(data_tensor.shape[0]*0.9)
val_size = data_tensor.shape[0] - train_size
train_data = data_tensor[:train_size]
val_data = data_tensor[train_size:]
assert train_data.shape[0] + val_data.shape[0] == data_tensor.shape[0]

In [65]:
torch.manual_seed(989)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[18, 18, 10,  0, 30, 39, 58, 41],
        [11,  0, 13, 52, 42,  6,  1, 39],
        [50,  1, 41, 53, 60, 43, 56,  5],
        [ 1, 58, 53, 52, 45, 59, 43,  1]])
targets:
torch.Size([4, 8])
tensor([[18, 10,  0, 30, 39, 58, 41, 50],
        [ 0, 13, 52, 42,  6,  1, 39, 57],
        [ 1, 41, 53, 60, 43, 56,  5, 42],
        [58, 53, 52, 45, 59, 43,  1, 42]])
----
when input is [18] the target: 18
when input is [18, 18] the target: 10
when input is [18, 18, 10] the target: 0
when input is [18, 18, 10, 0] the target: 30
when input is [18, 18, 10, 0, 30] the target: 39
when input is [18, 18, 10, 0, 30, 39] the target: 58
when input is [18, 18, 10, 0, 30, 39, 58] the target: 41
when input is [18, 18, 10, 0, 30, 39, 58, 41] the target: 50
when input is [11] the target: 0
when input is [11, 0] the target: 13
when input is [11, 0, 13] the target: 52
when input is [11, 0, 13, 52] the target: 42
when input is [11, 0, 13, 52, 42] the target: 6
when input is [11, 0

In [73]:
torch.manual_seed(1337)
class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.token_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx):
        logits = self.token_table(idx)
        return logits
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits = self.forward(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

In [74]:
torch.manual_seed(1337)
model = BigramModel(vocab_size)
res = model(xb)
res = res.view(res.size(0), res.size(2), res.size(1))
loss = F.cross_entropy(res, yb)
loss

tensor(4.6539, grad_fn=<NllLoss2DBackward0>)

In [79]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [87]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits= model(xb)
    logits = logits.view(logits.size(0), logits.size(2), logits.size(1))
    loss = F.cross_entropy(logits, yb)
    loss.backward()
    optimizer.step()

print(loss)

tensor(3.7726, grad_fn=<NllLoss2DBackward0>)


In [95]:
xb, _ = get_batch('val')
print(decode(model.generate(xb[0:1, :], max_new_tokens=500)[0].tolist()))

torch.Size([1, 8, 65])
torch.Size([1, 65])
torch.Size([1, 9, 65])
torch.Size([1, 65])
torch.Size([1, 10, 65])
torch.Size([1, 65])
torch.Size([1, 11, 65])
torch.Size([1, 65])
torch.Size([1, 12, 65])
torch.Size([1, 65])
torch.Size([1, 13, 65])
torch.Size([1, 65])
torch.Size([1, 14, 65])
torch.Size([1, 65])
torch.Size([1, 15, 65])
torch.Size([1, 65])
torch.Size([1, 16, 65])
torch.Size([1, 65])
torch.Size([1, 17, 65])
torch.Size([1, 65])
torch.Size([1, 18, 65])
torch.Size([1, 65])
torch.Size([1, 19, 65])
torch.Size([1, 65])
torch.Size([1, 20, 65])
torch.Size([1, 65])
torch.Size([1, 21, 65])
torch.Size([1, 65])
torch.Size([1, 22, 65])
torch.Size([1, 65])
torch.Size([1, 23, 65])
torch.Size([1, 65])
torch.Size([1, 24, 65])
torch.Size([1, 65])
torch.Size([1, 25, 65])
torch.Size([1, 65])
torch.Size([1, 26, 65])
torch.Size([1, 65])
torch.Size([1, 27, 65])
torch.Size([1, 65])
torch.Size([1, 28, 65])
torch.Size([1, 65])
torch.Size([1, 29, 65])
torch.Size([1, 65])
torch.Size([1, 30, 65])
torch.Size

In [58]:
math.log(1/65)

-4.174387269895637

In [97]:
decode(xb[0:1, :][0].tolist())

'nd that '